In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from pathlib import Path

# CONFIG
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS_HEAD = 10
LR_HEAD = 1e-3

DATASET_DIR = Path("/home/admin1/PROJETS_KEYCE/cc_transfert/dataset")


2026-02-06 11:59:12.423214: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-06 11:59:12.439248: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2026-02-06 11:59:12.464072: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2026-02-06 11:59:12.464119: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2026-02-06 11:59:12.479460: I tensorflow/core/platform/cpu_feature_gua

**Data generators**

In [2]:
train_gen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.05,
    height_shift_range=0.05,
    brightness_range=[0.8, 1.2],
    horizontal_flip=True
)

val_gen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

train_ds = train_gen.flow_from_directory(
    DATASET_DIR / "train",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

val_ds = val_gen.flow_from_directory(
    DATASET_DIR / "val",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

NUM_CLASSES = train_ds.num_classes
print("NUM_CLASSES =", NUM_CLASSES)


Found 1840 images belonging to 10 classes.
Found 466 images belonging to 10 classes.
NUM_CLASSES = 10


**Modèle MobileNetV2**

In [3]:
base_model = MobileNetV2(
    include_top=False,
    weights="imagenet",
    input_shape=(*IMG_SIZE, 3)
)
base_model.trainable = False


2026-02-06 11:59:15.639639: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2026-02-06 11:59:15.762816: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


**Classification**

In [4]:
inputs = layers.Input(shape=(*IMG_SIZE, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.4)(x)
outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

model = models.Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(LR_HEAD),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1280)           │         5,120 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,593,610 (9.89 MB)

 Trainable params: 333,066 (1.27 MB)

 Non-trainable params: 2,260,544 (8.62 MB)

**Entraînement**

In [5]:
history_head = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS_HEAD
)


Epoch 1/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 62s 992ms/step - accuracy: 0.5897 - loss: 1.4427 - val_accuracy: 0.7017 - val_loss: 0.9158
Epoch 2/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 53s 913ms/step - accuracy: 0.6875 - loss: 1.0269 - val_accuracy: 0.7575 - val_loss: 0.7602
Epoch 3/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 54s 931ms/step - accuracy: 0.7217 - loss: 0.8420 - val_accuracy: 0.7639 - val_loss: 0.6892
Epoch 4/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 53s 918ms/step - accuracy: 0.7582 - loss: 0.7525 - val_accuracy: 0.7790 - val_loss: 0.6691
Epoch 5/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 54s 936ms/step - accuracy: 0.7832 - loss: 0.6911 - val_accuracy: 0.7704 - val_loss: 0.6750
Epoch 6/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 53s 921ms/step - accuracy: 0.7978 - loss: 0.6069 - val_accuracy: 0.7575 - val_loss: 0.6725
Epoch 7/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 54s 935ms/step - accuracy: 0.7837 - loss: 0.6226 - val_accuracy: 0.7940 - val_loss: 0.6532
Epoch 8/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 53s 920ms/step - accuracy: 0.8005 - loss: 0.5945 - val_accu

**Sauvegarde du modèle**

In [6]:
model.save("mobilenetv2_head.h5")
print("✅ MobileNetV2 head sauvegardé")

✅ MobileNetV2 head sauvegardé


**Dégel partiel contrôlé**

In [7]:
# Nombre de couches à dégeler
FINE_TUNE_AT = len(base_model.layers) - 30
for layer in base_model.layers[:FINE_TUNE_AT]:
    layer.trainable = False
for layer in base_model.layers[FINE_TUNE_AT:]:
    if not isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = True
print(f"Couches entraînables : {sum(l.trainable for l in base_model.layers)}")


Couches entraînables : 19


**Recompile (LR faible)**

In [8]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

**Fine-tuning**

In [9]:
EPOCHS_FINE = 10

history_fine = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS_FINE
)


Epoch 1/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 63s 991ms/step - accuracy: 0.7571 - loss: 0.7077 - val_accuracy: 0.4742 - val_loss: 3.0251
Epoch 2/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 55s 955ms/step - accuracy: 0.7598 - loss: 0.6993 - val_accuracy: 0.6159 - val_loss: 1.8743
Epoch 3/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 54s 932ms/step - accuracy: 0.7859 - loss: 0.6034 - val_accuracy: 0.5751 - val_loss: 1.4182
Epoch 4/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 55s 944ms/step - accuracy: 0.8016 - loss: 0.5717 - val_accuracy: 0.6352 - val_loss: 1.2661
Epoch 5/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 55s 948ms/step - accuracy: 0.8033 - loss: 0.5783 - val_accuracy: 0.5858 - val_loss: 1.7028
Epoch 6/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 55s 939ms/step - accuracy: 0.8288 - loss: 0.5080 - val_accuracy: 0.7275 - val_loss: 1.2344
Epoch 7/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 53s 917ms/step - accuracy: 0.8147 - loss: 0.5253 - val_accuracy: 0.3348 - val_loss: 7.8922
Epoch 8/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 54s 928ms/step - accuracy: 0.8326 - loss: 0.4756 - val_accu

**Sauvegarde finale**

In [10]:
model.save("mobilenetv2_finetuned.h5")
print("✅ MobileNetV2 fine-tuned sauvegardé")

✅ MobileNetV2 fine-tuned sauvegardé


**Dégel partiel contrôlé (30 dernières couches)**

In [11]:
# Nombre total de couches du backbone
total_layers = len(base_model.layers)
print("Total layers in base model:", total_layers)

# On dégèle les 30 dernières couches
FINE_TUNE_AT = total_layers - 30

for layer in base_model.layers[:FINE_TUNE_AT]:
    layer.trainable = False

for layer in base_model.layers[FINE_TUNE_AT:]:
    if not isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = True

# Vérification
trainable_count = sum(layer.trainable for layer in base_model.layers)
print("Couches entraînables:", trainable_count)


Total layers in base model: 154
Couches entraînables: 19


**Recompile**

In [12]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

**Callbacks**

In [13]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=4,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.3,
        patience=2,
        min_lr=1e-6
    )
]

**Évaluation finale**

In [14]:
val_loss, val_acc = model.evaluate(val_ds)
print(f"Validation accuracy finale : {val_acc:.4f}")

15/15 ━━━━━━━━━━━━━━━━━━━━ 10s 482ms/step - accuracy: 0.5751 - loss: 1.4750
Validation accuracy finale : 0.5751


**Sauvegarde du modèle (PROPRE)**

In [15]:
MODEL_PATH = "mobilenetv2_gc10_finetuned.h5"
model.save(MODEL_PATH)

print("✅ Modèle sauvegardé :", MODEL_PATH)

✅ Modèle sauvegardé : mobilenetv2_gc10_finetuned.h5


**Sauvegarde des métadonnées**

In [16]:
import json

metadata = {
    "model": "MobileNetV2",
    "img_size": IMG_SIZE,
    "num_classes": NUM_CLASSES,
    "val_accuracy": float(val_acc)
}

with open("mobilenetv2_metadata.json", "w") as f:
    json.dump(metadata, f, indent=4)

print("📦 Métadonnées sauvegardées")


📦 Métadonnées sauvegardées
